## Test AI Angent Work flow for Bank Statements

In [4]:
import os, json, time, gc

from dotenv import load_dotenv
from IPython.display import HTML, Markdown, Image, Video
from tqdm import tqdm
from openai import OpenAI, AsyncOpenAI
import asyncio
import aiohttp
import pandas as pd

#fix bug with aysncio and jupyter
import nest_asyncio # for langchain async 
nest_asyncio.apply()

import numpy as np

/home/johnsonhk88/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [5]:
class CFG:
    OFFLINE = False #True # for Test offline environment
    USE_LLAMA3 = False # 
    USE_GEMMA2 = False # 
    USE_QWEN = False # 
    USE_DEEPSEEK = True # 
    USE_DEEPSCALE = False # 

    TASK_GEN = True # for generative Text output task (suitable for RAG project)
    TEST_LLM = True
    USE_HUGGINGFACE = True # Pull model from Huggingface model hub
    USE_LMSTUIDO = False # for local LLM framework 
    USE_OLLAMA = False # for OLLAMA local LLM framework 
    USE_VLLM = False # for VLLM  LLM framework

    # mulitlingual LLM model 
    model1 = "meta-llama/Llama-3.2-3B-Instruct"  # llama3.2  3B-Instruct

    model2 =  "google/gemma-2-2b-it" # gemma 2 9B (mulitlingual)
    model3 = "Qwen/Qwen2.5-3B-Instruct" # Qwen 3B (mulitlingual)
    model4 = 'Qwen/Qwen2.5-7B-Instruct' # Qwen 7B (mulitlingual)
    model5 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B" # DeepSeek Distill 1.5B (mulitlingual)
    model6 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B" # DeepSeek Distill 7B (mulitlingual)
    model7 = "agentica-org/DeepScaleR-1.5B-Preview"

    # for VLM model
    vlmModel1 = "Qwen/Qwen2.5-VL-3B-Instruct"
    vlmModel2 = "Qwen/Qwen2.5-VL-7B-Instruct"


    # Mult Embedding model
    embedModel1 = 'intfloat/multilingual-e5-small' # for embedding model support chinese
    embedModel2 = "intfloat/multilingual-e5-large-instruct"
    embedModel3 = "Alibaba-NLP/gte-Qwen2-1.5B-instruct" # for embedding model support chinese
    embedModel4 = "Alibaba-NLP/gte-multilingual-base" # for embedding model support chinese
    embedModel5 = "BAAI/bge-m3" # for multilingual embedding model
    embedModel6 = "jinaai/jina-embeddings-v3"
    

    # LLM for AI Agent 
    llmModel1 = "ollama/deepseek-r1:8b"
    llm_base_url1 = "http://localhost:11434"


    FEW_SHOT_TEST= False#True
    USE_WANDB = True#True # for  LLM evalution and debug , track fine tuning performance

    USE_DEEPEVAL = True#False # for LLM evalution   
    USE_TRAIN =  False #True #False#True Much be use GPU for Training 
    
    # For VectorDB selection
    USE_FAISS = False#True # For RAG VectorDB
    USE_CHROMA = True #False #True #False # for RAG VectorDF
    USE_PINECONE = False#True#False #True # for RAG VectorDF
    USE_WEAVIATE = False#True #False # for RAG VectorDF
    USE_MILVUS = False#True              # for RAG VectorDF

    # for LLM fine tuning
    maxTrainData = 200#3500#5000 #10000#5000 #10000
    maxEvalData = 20#100 # 20 


    # LLM parameters
    reportTo ="none"
    topK = 40
    topP = 1.0
    temperature = 0.6 #0.5
    repetition_penalty = 1.05 # 1.1
    maxOutToken = 1024#180 #100
    

    
    maxToken=  512#768#512#768 # 512 for test only

In [6]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai.tools import BaseTool, tool
from crewai import LLM
import crewai
from pydantic import BaseModel, Field
from typing import List, Dict, Type
from crewai_tools import PDFSearchTool

/home/johnsonhk88/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/lib/python3/dist-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 45.0.0.
  "class": algorithms.Blowfish,
/usr/lib/python3/dist-packages/paramiko/tra

## CrewAI Support LLM List :  LiteLLM 
- <https://docs.crewai.com/how-to/llm-connections>
- <https://docs.crewai.com/concepts/llms>



In [ ]:
#

In [7]:
llm = LLM(
    model= CFG.llmModel1,
    base_url= CFG.llm_base_url1,
)

In [8]:
# 
document_agent = Agent( role="Extract Document Agent",
    goal="Extract information from a document",
    backstory="You are a document extraction agent. You will be given a document and you need to extract the relevant information from it.",
    # tools=[webSearchTool],
    llm=llm,
    max_iter=10,
    allow_delegation= True,
    verbose=True,

)


In [9]:
rag_agent = Agent( role="RAG Agent",
                  goal="Extract information from a document",
                 backstory=("You are a document retrieval agent."
                              "You will be given a document and you need to extract the relevant information from it."),
                 llm=llm,
                 allow_delegation= True,
                 verbose=True,
)

In [11]:
fiancial_agent = Agent( role= "Personal Finance Analystic Expert",
                        goal="Extract information from a document",
                        backstory=("You are a document retrieval agent."
                                   "You will be given a document and you need to extract the relevant information from it."),
                        # tools=[webSearchTool],
                        llm=llm,
                        max_iter=10,
                        allow_delegation= True,
                        verbose=True,
)

In [12]:
report_agent = Agent( role= "Report Expert for Financial Analystic",
                      goal="",
                      backstory=(""),
                     
                     )



